In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import os

Data sources
- https://www.fao.org/faostat/en/#data/RFN

In [2]:

## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [3]:
# Find and read the CSV file from the raw_data folder
target_file = 'FAOSTAT_data_2-20-2023.csv'
target_file_path = os.path.join(raw_data_path, target_file)

In [4]:
fao = pd.read_csv(target_file_path)
fao.drop(columns=['Unnamed: 1'], inplace=True)
dtype_dict = {}
for col in fao.columns[2:]:
    dtype_dict[col] = float

In [5]:
fao = pd.read_csv(target_file_path, dtype=dtype_dict, na_values=['Value'])
fao

,Unnamed: 0,Unnamed: 1,Unnamed: 2,1961,1962,1963,1964,1965,1966,1967,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Production,Nutrient nitrogen N (total),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,17555.0,15776.0,15694.0,14416.0,14416.0,14416.0,14416.0,14416.0,14416.0,14416.0
2,Afghanistan,Production,Nutrient phosphate P2O5 (total),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1563.0,460.0,1539.0,1541.0,1541.0,1541.0,1541.0,1541.0,1541.0,1541.0
3,Afghanistan,Production,Nutrient potash K2O (total),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Albania,Production,Nutrient nitrogen N (total),NaN,NaN,NaN,NaN,NaN,NaN,5000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,Zambia,Production,Nutrient phosphate P2O5 (total),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
522,Zambia,Production,Nutrient potash K2O (total),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
523,Zimbabwe,Production,Nutrient nitrogen N (total),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,28000.0,20300.0,22200.0,25700.0,29300.0,11700.0,12300.0,13200.0,13200.0,13200.0
524,Zimbabwe,Production,Nutrient phosphate P2O5 (total),11000.0,11000.0,15000.0,15000.0,20000.0,20000.0,25500.0,...,0.0,0.0,5600.0,3800.0,2000.0,2100.0,9500.0,8100.0,8100.0,8100.0


In [6]:
fao = pd.read_csv(target_file_path)
fao.drop(0, inplace=True)
fao.drop(columns=['Unnamed: 1'], inplace=True)
fao.dropna(how='all', inplace=True)
iso2 = []
for country in fao['Unnamed: 0']:
    if country=='China, Taiwan Province of':
        iso2.append('TW')
    elif country=='Serbia and Montenegro':
        iso2.append('CS')
    elif country=='USSR':
        iso2.append('SU')
    elif country=='Yugoslav SFR':
        iso2.append('YU')
    elif country=='Czechoslovakia':
        iso2.append('CSK')   
    else:
        iso2.append(coco.convert(names=country, to='iso2'))
fao['Country Code'] = iso2
fao.rename(columns={'Unnamed: 0': 'Country Name', 'Unnamed: 2': 'Technology Name'}, inplace=True)
fao['Unit'] = 'metric tons'
fao['Data Source'] = 'FAOSTAT'
fao['Spatial Scale'] = 'National'
fao['Metric'] = 'Annual Production'
fao.replace('Nutrient nitrogen N (total)', 'Nitrogen Fertilizer', inplace=True)
fao.replace('Nutrient phosphate P2O5 (total)', 'Phosphate Fertilizer', inplace=True)
fao.replace('Nutrient potash K2O (total)', 'Potash Fertilizer', inplace=True)
fao['ID'] = fao['Technology Name'] + '_' + fao['Metric'] + '_' + fao['Country Code']
fao.set_index('ID', inplace=True)
omit = []
for col in fao.columns:
    if len(col) != 4 or col == 'Unit':
        omit.append(col)
empty_rows = fao.drop(columns=omit)
empty_rows.dropna(how='all', inplace=True)
na_idx = []
for country in fao.index:
    if country not in empty_rows.index:
        na_idx.append(country)
fao.drop(na_idx, inplace=True)
fao

Belgium-Luxembourg not found in regex
Belgium-Luxembourg not found in regex
Belgium-Luxembourg not found in regex


,Country Name,Technology Name,1961,1962,1963,1964,1965,1966,1967,1968,...,2016,2017,2018,2019,2020,Country Code,Unit,Data Source,Spatial Scale,Metric
ID,,,,,,,,,,,,,,,,,,,,,
Nitrogen Fertilizer_Annual Production_AF,Afghanistan,Nitrogen Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,14416,14416,14416,14416,14416,AF,metric tons,FAOSTAT,National,Annual Production
Phosphate Fertilizer_Annual Production_AF,Afghanistan,Phosphate Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1541,1541,1541,1541,1541,AF,metric tons,FAOSTAT,National,Annual Production
Potash Fertilizer_Annual Production_AF,Afghanistan,Potash Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,AF,metric tons,FAOSTAT,National,Annual Production
Nitrogen Fertilizer_Annual Production_AL,Albania,Nitrogen Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,5000,17000,...,0,0,0,0,0,AL,metric tons,FAOSTAT,National,Annual Production
Phosphate Fertilizer_Annual Production_AL,Albania,Phosphate Fertilizer,NaN,NaN,NaN,500,2000,3000,5000,14000,...,0,0,0,0,0,AL,metric tons,FAOSTAT,National,Annual Production
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Phosphate Fertilizer_Annual Production_ZM,Zambia,Phosphate Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,ZM,metric tons,FAOSTAT,National,Annual Production
Potash Fertilizer_Annual Production_ZM,Zambia,Potash Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,ZM,metric tons,FAOSTAT,National,Annual Production
Nitrogen Fertilizer_Annual Production_ZW,Zimbabwe,Nitrogen Fertilizer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11700,12300,13200,13200,13200,ZW,metric tons,FAOSTAT,National,Annual Production


In [7]:
# drop mainland China (near duplicate of China, which has slightly higher numbers for some years)
# drop Sudan (former), dup
# drop Belgium-Luxembourg (not regional data)
#fao = fao[(fao['Country Name'] != 'China, mainland') & (fao['Country Name'] != 'Sudan (former)') & (fao['Country Name'] != 'Belgium-Luxembourg')]
#fao

Save file

In [8]:

output_file = 'fao.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)

fao.to_csv(output_file_path)
print("Data saved to:", output_file_path)


Data saved to: /Users/jennagreene/Documents/GitHub/HATCH_data/cleaned_data/fao.csv
